In [29]:
import os
import datetime
import numpy as np
import cv2
import natsort


# # # # # # # # # # # # # # # # # # # # # # Normalized implement is necessary part as the fore-processing   # # # # # # # # # # # # # # # #
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
            
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

from extras.DetermineDepth import determineDepth
from extras.TransmissionEstimation import getTransmission

from extras.getAdaptiveSceneRadiance import AdaptiveSceneRadiance
from extras.getAtomsphericLight import getAtomsphericLight
#from extras.refinedTransmission import refinedtransmission

from extras.sceneRadianceGb import sceneRadianceGB
from extras.sceneRadianceR import sceneradiance
from extras.GuidedFilter import GuidedFilter
from extras.guidedfilter_he import guided_filter_he
#from extras.getAdaptiveExposureMap import AdaptiveExposureMap

In [30]:
def  refinedtransmission(transmission, img):
    gimfiltR = 50  # 引导滤波时半径的大小
    eps = 10 ** -3  # 引导滤波时epsilon的值

    guided_filter = GuidedFilter(img, gimfiltR, eps)
    transmission[:,:,0] = guided_filter.filter(transmission[:,:,0])
    transmission[:,:,1] = guided_filter.filter(transmission[:,:,1])
    transmission = np.clip(transmission, 0.1, 0.9)


    return transmission

In [31]:
def AdaptiveExposureMap(img, sceneRadiance, Lambda, blockSize):

    minValue = 10 ** -2
    img = np.uint8(img)
    sceneRadiance = np.uint8(sceneRadiance)

    YjCrCb = cv2.cvtColor(sceneRadiance, cv2.COLOR_BGR2YCrCb)
    YiCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    normYjCrCb = (YjCrCb - YjCrCb.min()) / (YjCrCb.max() - YjCrCb.min())
    normYiCrCb = (YiCrCb - YiCrCb.min()) / (YiCrCb.max() - YiCrCb.min())
    Yi = normYiCrCb[:, :, 0]
    Yj = normYjCrCb[:, :, 0]
    Yi = np.clip(Yi, minValue,1)
    Yj = np.clip(Yj, minValue,1)

    # print('np.min(Yi)',np.min(Yi))
    # print('np.max(Yi)',np.max(Yi))
    # print('np.min(Yj)',np.min(Yj))
    # print('np.max(Yj)',np.max(Yj))
    # Yi = YiCrCb[:, :, 0]
    # Yj = YjCrCb[:, :, 0]
    S = (Yj * Yi + 0.3 * Yi ** 2) / (Yj ** 2 + 0.3 * Yi ** 2)

    # print('S',S)

    gimfiltR = 50  # 引导滤波时半径的大小
    eps = 10 ** -3  # 引导滤波时epsilon的值

    # refinedS = guided_filter_he(YiCrCb, S, gimfiltR, eps)

    guided_filter = GuidedFilter(YiCrCb, gimfiltR, eps)
    # guided_filter = GuidedFilter(normYiCrCb, gimfiltR, eps)

    refinedS = guided_filter.filter(S)

    # print('guided_filter_he(YiCrCb, S, gimfiltR, eps)', refinedS)
    # S = np.clip(S, 0, 1)

    # cv2.imwrite('OutputImages_D/' + 'SSSSS' + '_GBdehazingRCorrectionStretching.jpg', np.uint8(S * 255))

    S_three = np.zeros(img.shape)
    S_three[:, :, 0] = S_three[:, :, 1] = S_three[:, :, 2] = refinedS

    return S_three

In [10]:
print(os.path.abspath("."))

C:\Users\Giridhar\Downloads\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Underwater Image Color Restoration\GBdehazingRCorrection


In [32]:
np.seterr(over='ignore')
if __name__ == '__main__':
    pass

starttime = datetime.datetime.now()

# folder = "C:/Users/Administrator/Desktop/UnderwaterImageEnhancement/Physical/GBdehazingRCorrection"
folder = (os.path.abspath("."))
path = folder + "/Datasets"
files = os.listdir(path)
files =  natsort.natsorted(files)

for i in range(len(files)):
    file = files[i]
    filepath = path + "/" + file
    prefix = file.split('.')[0]
    if os.path.isfile(filepath):
        print('********    file   ********',file)
        img = cv2.imread(folder +'/Datasets/' + file)
        img = (img - img.min()) / (img.max() - img.min()) * 255
        blockSize = 9
        largestDiff = determineDepth(img, blockSize)
        AtomsphericLight, AtomsphericLightGB, AtomsphericLightRGB = getAtomsphericLight(largestDiff, img)
        print('AtomsphericLightRGB',AtomsphericLightRGB)
        
        # transmission = getTransmission(img, AtomsphericLightRGB, blockSize=blockSize)
        transmission = getTransmission(img, AtomsphericLightRGB, blockSize)
        # print('transmission.shape',transmission.shape)
        # TransmissionComposition(folder, transmission, number, param='coarse')
        transmission = refinedtransmission(transmission, img)

        cv2.imwrite(folder + prefix + '_GBDehazedRcoorectionUDCP_TM.jpg', np.uint8(transmission[:, :, 0] * 255))
        

        # TransmissionComposition(folder, transmission, number, param='refined_15_175_175')
        sceneRadiance_GB = sceneRadianceGB(img, transmission, AtomsphericLightRGB)

        # cv2.imwrite('OutputImages/' + prefix + 'GBDehazed.jpg', sceneRadiance_GB)


        # # print('sceneRadiance_GB',sceneRadiance_GB)
        sceneRadiance = sceneradiance(img, sceneRadiance_GB)
        # sceneRadiance= sceneRadiance_GB
        # cv2.imwrite('OutputImages/'+ prefix + 'GBDehazedRcoorectionUDCP.jpg', sceneRadiance)
        # # print('np.min(sceneRadiance)',np.min(sceneRadiance))
        # # print('sceneRadiance',sceneRadiance)
        
        S_x = AdaptiveExposureMap(img, sceneRadiance, Lambda=0.3, blockSize=blockSize)
        # print('S_x',S_x)
        sceneRadiance = AdaptiveSceneRadiance(sceneRadiance, S_x)
        outputpath = folder 
        # print('sceneRadiance',sceneRadiance)
        cv2.imwrite( outputpath + prefix + 'GBDehazedRcoorectionUDCPAdaptiveMap.jpg', sceneRadiance)
        if not cv2.imwrite(outputpath + prefix + 'GBDehazedRcoorectionUDCPAdaptiveMap.jpg', sceneRadiance):
            raise Exception("Could not write image")
        


Endtime = datetime.datetime.now()
Time = Endtime - starttime
print('Time', Time)

********    file   ******** 1.png
AtomsphericLightRGB [ 45. 154.  33.]
***********************************************************
parameterR 4.896559188612946
********    file   ******** 2.png
AtomsphericLightRGB [137. 214.  78.]
***********************************************************
parameterR 4.465319200527943
********    file   ******** 3.png
AtomsphericLightRGB [ 72. 154.   3.]
***********************************************************
parameterR 119.0614132490574
********    file   ******** 4.png
AtomsphericLightRGB [ 93.29269  145.12195    8.292683]
***********************************************************
parameterR 14.084395847527253
********    file   ******** 5.png
AtomsphericLightRGB [ 73. 200.  88.]
***********************************************************
parameterR 4.272588416806484
********    file   ******** 16.png
AtomsphericLightRGB [186. 174.   3.]
***********************************************************
parameterR 64.8777880556569
********    file   **

In [34]:
print(folder)
print(path)
print(folder + '\\OutputImages\\')
print(outputpath)

C:\Users\Giridhar\Downloads\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Underwater Image Color Restoration\GBdehazingRCorrection
C:\Users\Giridhar\Downloads\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Underwater Image Color Restoration\GBdehazingRCorrection/Datasets
C:\Users\Giridhar\Downloads\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Underwater Image Color Restoration\GBdehazingRCorrection\OutputImages\
C:\Users\Giridhar\Downloads\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Single-Underwater-Image-Enhancement-and-Color-Restoration-master\Underwater Image Color Restoration\GBdehazingRCorrection
